In [1]:
import LemurBounce2
from LFOFM import *
import PydalChanel as pydalModule

l = LemurBounce2.LemurBounce2()

gateKeys = []

def saveWorld(key):
    l.sendOSCMessage('/saveWorld', 1, key)

def loadWorld(key):
    l.sendOSCMessage('/loadWorld', 1, key)
    
def setGate(key, m, b):
    if not key in gateKeys:
        gateKeys.append(key)
        l.sendOSCMessage('/setGate', 1, [key, m, b])
    
def toggleGate(key):
    if key in gateKeys: 
        l.sendOSCMessage('/toggleGate', 1, key)

def remapBalls(ballList):
    l.sendOSCMessage('/changeBallDeviation', 0, ballList)

def remapBallChan(ballList):
    l.sendOSCMessage('/changeBallChan', 0, ballList)

def remapGates(noteList):
    l.sendOSCMessage('/changeGateNotes', 0, noteList)

def remapBallDecay(noteList):
    l.sendOSCMessage('/changeBallDecay', 0, noteList)

def runFunc(delayVal, oscEndpoint, func):
    l.superColliderServer.addMsgHandler("/"+oscEndpoint, func)
    l.sendOSCMessage("/triggerDelayFunc", 0, [delayVal, "/"+oscEndpoint])
    
def friction(val):
    l.sendOSCMessage("/setParam", 1, ['friction', val]) 

def slowDown(num):
    friction(0.5)
    runFunc(num, "fric"+str(num),  lambda a, b, c, d: friction(0))

def speedMult(val):
    l.sendOSCMessage("/setParam", 1, ['speed', val]) 

def chord(chordDef):
    return lambda : remapGates(chordDef)

def chordSeq(chordSeq):
    st = 'abcdefghijk'
    for i in range(len(chordSeq)):
        funcMap[st[i]] = chord(chordSeq[i])
    
pydal = pydalModule.Pydal()
ch1 = pydal.newChannel(1)
read = pydalModule.read

funcMap = l.triggerFunctions

wp = WavePlayer()

In [22]:
# gateToNote = [60, 64, 67, 79, 60, 60, 60, 60, 60];
# ballToDeviation = [-3, 5, -4, 9];


OSCServer: NameError on request from localhost:57120: global name 'remapChord' is not defined


In [ ]:
speedMult(.25)

In [ ]:
remapBallChan([3]*4)
remapBallDecay([0.01]*4)

In [13]:
cMaj = [60,64,67,72]
dMin = [57,62,65,69]
g7 = [55,59,62,65]
fMaj = [53,57,60,65]
d7 = [54,57,60,62]

someFifth = [0,5,0,0]

remapGates(cMaj)
remapBalls(someFifth)
setGate(1, 0, 700)
setGate(2, 0, 500)
setGate(3, 0, 300)
setGate(4, 0, 100)

In [23]:
chordSeq([cMaj, dMin, g7, fMaj, d7])
ch1.play(read("a b a d b c a e", 32, 'funcTrigger'))

In [ ]:
l.sendOSCMessage("/slingshot", 1, [0, 7, 2, 0, 30, [1]])
l.sendOSCMessage("/slingshot", 1, [0, 7, 4, 0, 50, [2]])
l.sendOSCMessage("/slingshot", 1, [0, 7, 8, 0, 30, [3]])

In [ ]:
wp.lemurWave(Sin(), 0)

In [ ]:
runFunc(10, "test", lambda a, b, c, d: remapGates([41, 62, 63, 64]))

In [ ]:
loadWorld("a")

In [ ]:
ch1.play(read('a c b a b d', 8, 'ballState'))

In [ ]:
def funca():
    remapBalls([-10, -3, 4, 10])
def funcb():
    remapBalls([0, 3, 7, 10])
def funcc():
    remapBalls([0, 3, 6, 9])
l.triggerFunctions["a"] = funca
l.triggerFunctions["b"] = funcb
l.triggerFunctions["c"] = funcc

In [ ]:
ch1.play(read('a c b a b c', 32, 'funcTrigger'))
# ch1.stop()

In [ ]:
def funcd():
    toggleGate(1)
    toggleGate(2)
def funce():
    toggleGate(3)
    toggleGate(4)

l.triggerFunctions["d"] = funcd
l.triggerFunctions["e"] = funce

In [ ]:
# ch2.play(read('d e d d e f', 1, 'funcTrigger'))
ch2.stop()
ch1.stop()

How can we have "musical" interactions with this state-replay mechanism?
- keep a running buffer of saved state every 16th or 32nd note. you can take "snapshots" of this running buffer. You can treat these snapshots as "notes" and resequence (like with tidalcycles) or reshuffle them. Experiment with using this technique for longer intervals. This is mainly to impose structure in worlds that are very chaotic. 
- In worlds that are less chaotic, and where user interaction changes some "parts" of the sound but also leaves most of the "parts" unchanged, you can get away with less frequent, more strategic changes. 
- goal - define worlds where the above type of "interaction by parts" is possible
- save/reload only subsets of the bodies, transform their positions and momentums. this would be especially useful in the "infinte non collision" world. 
- map movements/collions to "higher level" musical events, and deal with less objects in the world. 
- In general, we want to be able to modify the world with intentionality - meaning have a sense of what will be changed/preserved musically as the result of an interaction. 


- having the live-coder do something truly "procedural" seems like it would (usually) be too complicated for the non-coder to figure out during performance. However, this still leaves the relative strengths of "future scheduling" and "dense structure modification" to exploit
    - there are exceptions, but you'll have to work hard to find them. And it would help if the "procedure" or "rules" were minimal (but had interesting emergent properties), and there was a way to visualize/describe them succinctly for the non-coder. Some way to procedurally define how elements disapear/reappear based on their interactions would be cool   
    
    
- general feature - schedule some parameter to change (or really, some function to execute) in the future, and notify the non-coder that that event is going to happen and give some visual countdown to it. Can just show a text string to describe the event that will happen

- need enough things to do in the system (or in any real-time collaborative system) so that the live coder and non coder are not modifying the same "part of the world" at the same time. You want to enable a collaborative building metaphor where they can work in paralellel. The strengths of such a system are that both live coder and non coder can manipulate the same "world" in different ways, but not having enough stuff to manipulate forces them to start manipulating the same things at the same time to stay engaged, and then they start stepping on each others toes and have to start reacting to each other at a very short time scale, which can be difficult. 

.  
.  
.  
.  
.  
late night mumbo jumbo
- We still need to give the live-coder something to do - some procedure to define rather than some parameters to set. 
- In general, the role of the live-coder in these systems is not just to define/modify processes/strucutres, but to remap abstract processes/structures to musical semantics. 
    - for example - you could have some sort of networked gravity system (n-body) where balls are attracted by each-other and bounce off of walls. However, one ball is heavier than the rest and the center of the system (each ball corresponds to a note, and a ball/wall colision plays that note). The live-coder can not only play with the physics parameters of the system, but can remap what note corresponds to what ball (musical semantics). In particular, changing what note is mapped to the central ball can effectively change the tonality of the music being generated. 
    - hypothesis - it is (ususally) easier 